## Импорт библиотек

In [1]:
import pandas as pd
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langchain_huggingface import HuggingFaceEmbeddings
import torch
import tqdm as notebook_tqdm
import os
from langchain_community.document_loaders import Docx2txtLoader, PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_qdrant import QdrantVectorStore
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader

from langchain_community.document_loaders import PyMuPDFLoader
from tqdm import tqdm
import time
import unstructured

from langchain_community.document_loaders import UnstructuredWordDocumentLoader
import pikepdf

## Инициализация БД Qdrant с коллекцией

In [4]:
client = QdrantClient(
    path='../db/qdrant_db',
    )

In [3]:
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='norma_ai_collection'), CollectionDescription(name='norma_ai_collection_ru-en-RoSBERTa')])

In [104]:
client.delete_collection("norma_ai_collection_ru-en-RoSBERTa")

True

In [112]:
collection_1 = 'norma_ai_collection_ru-en-RoSBERTa'
collection_2 = 'norma_ai_collection'

In [105]:
client.create_collection(
    collection_name="norma_ai_collection_ru-en-RoSBERTa",
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
)

True

## Инициализация модели для векторизации

In [ ]:
model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
model_kwargs = {'device': 'cuda' if torch.cuda.is_available() else 'cpu'}
encode_kwargs = {'normalize_embeddings': True, 'batch_size':128}

embeddings_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

In [99]:
# ai-forever/ru-en-RoSBERTa
model_name_2 = "ai-forever/ru-en-RoSBERTa"
model_kwargs_2 = {'device': 'cuda' if torch.cuda.is_available() else 'cpu'}
encode_kwargs_2 = {'normalize_embeddings': True, 'batch_size':128}

embeddings_model_2 = HuggingFaceEmbeddings(
    model_name=model_name_2,
    model_kwargs=model_kwargs_2,
    encode_kwargs=encode_kwargs_2,
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/241 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.61G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

## Загрузка датасета

In [4]:
all_documents = os.listdir('../data')

In [5]:
all_documents

['17. Приказ РТН № 567.pdf',
 'ГОСТ Р 55990-2014.pdf',
 '14. Приказ МЧС России № 302.pdf',
 'ГОСТ Р 2.601-2019.docx',
 'ГОСТ 31294-2005.docx',
 'ГОСТ Р 58329-2018.docx',
 '5. ФЗ 184 О техническом регулировании.pdf',
 '3. ТР ТС 012-2011.pdf',
 '4. ТР ТС 032-2013 О безопасности оборудования, работающего под избыточным давлением.docx',
 'Постановление Правительства РФ от 17.08.2020 N 1243.pdf',
 '9. ФЗ 384 О безопасности зданий и сооружений.docx',
 '17. Приказ РТН № 533 ФНИП.docx',
 '10. Постановление Правительства № 1437 Об утверждении Положения о разработке ПМЛЛПА на ОПО.docx',
 'ГОСТ 31385-2016 Резервуары вертикальные цилиндрические стальные.docx',
 '4.0. ФЗ 7 Об охране окружающей среды.pdf',
 'СТО 9.3-011-2011 Ингибиторная защита от коррозии.pdf',
 '16. Приказ РТН № 528 ФНИП Правила безопасного ведения газоопасных, огневых и ремонтных работ.docx',
 'СТО Газпром НТП 1.8-001-2004.pdf',
 '8.1. ФЗ 190 Градостроительный кодекс РФ.docx',
 '17. Приказ РТН № 420 ФНИП Правила проведения экспер

In [50]:
all_documents = os.listdir('../data')
paths = '/srv/norma_ai/norma_ai_project/data/'

loaded_docs = []
broken_files = []

def repack_pdf(input_path, output_path):
    with pikepdf.open(input_path) as pdf:
        pdf.save(output_path)

for i, p in enumerate(tqdm(all_documents, desc="Enumerating")):
    print(f'Файл № {i+1}')
    current_path = paths + p
    print(f"Trying to load file: {p}")
    if current_path.lower().endswith(".docx"):
        try:
            loaded_docs.extend(Docx2txtLoader(current_path).load())
            print(f"The download was successful: {p}")
            print('-'*50)
        except Exception as e:
            print(f'Нерабочий файл: {p}')
            broken_files.append(p)
            print('-'*50)
            continue
    elif current_path.lower().endswith(".pdf"):
        path = Path(current_path)
        try:
            loaded_docs.extend(PDFPlumberLoader(str(path)).load())
            print(f"The download was successful: {p}")
            print('-'*50)
        except Exception as e:
            print(f'Нерабочий файл: {p}')
            broken_files.append(p)
            print('-'*50)
            continue
    else:
        broken_files.append(p)
            

Enumerating:   0%|          | 0/45 [00:00<?, ?it/s]

Файл № 1
Trying to load file: 19. Приказ РТН № 536 ФНИП Правила промышленной безопасности при использовании оборудования, работающего под избыточным давлением.docx


Enumerating:   2%|▏         | 1/45 [00:00<00:10,  4.14it/s]

The download was successful: 19. Приказ РТН № 536 ФНИП Правила промышленной безопасности при использовании оборудования, работающего под избыточным давлением.docx
--------------------------------------------------
Файл № 2
Trying to load file: 17. Приказ РТН № 567.pdf


Enumerating:   7%|▋         | 3/45 [00:01<00:13,  3.19it/s]

...

The download was successful: 17. Приказ РТН № 136.pdf
--------------------------------------------------
Файл № 38
Trying to load file: СП 68.13330.2017 Приемка в эксплуатацию законченных строительство объектов.docx
The download was successful: СП 68.13330.2017 Приемка в эксплуатацию законченных строительство объектов.docx
--------------------------------------------------
Файл № 39
Trying to load file: 17. Приказ РТН № 450 Об утверждении Руководства по безопасности факельных систем.pdf


Enumerating:  93%|█████████▎| 42/45 [02:07<00:04,  1.57s/it]

The download was successful: 17. Приказ РТН № 450 Об утверждении Руководства по безопасности факельных систем.pdf
--------------------------------------------------
Файл № 40
Trying to load file: ГОСТ Р 58972-2020 Общие правила отбора образцов для испытаний продукции при подтверждении соответствия.docx
The download was successful: ГОСТ Р 58972-2020 Общие правила отбора образцов для испытаний продукции при подтверждении соответствия.docx
--------------------------------------------------
Файл № 41
Trying to load file: ГОСТ Р 59652-2021 Футеровки теплового оборудования. Основные требования.docx
The download was successful: ГОСТ Р 59652-2021 Футеровки теплового оборудования. Основные требования.docx
--------------------------------------------------
Файл № 42
Trying to load file: 21. Приказ РТН № 330.docx
The download was successful: 21. Приказ РТН № 330.docx
--------------------------------------------------
Файл № 43
Trying to load file: 7. ФЗ 116 О промышленной безопасности ОПО Тихомир

Enumerating: 100%|██████████| 45/45 [02:33<00:00,  3.41s/it]

The download was successful: 7. ФЗ 116 О промышленной безопасности ОПО Тихомирова.pdf
--------------------------------------------------
Файл № 44
Trying to load file: 2. ТР ТС 010-2011 О безопасности машин и оборудования.docx
The download was successful: 2. ТР ТС 010-2011 О безопасности машин и оборудования.docx
--------------------------------------------------
Файл № 45
Trying to load file: ГОСТ Р 57955-2017 Здания и сооружения газонефтедобывающих производств. Нормы проектирования.docx
The download was successful: ГОСТ Р 57955-2017 Здания и сооружения газонефтедобывающих производств. Нормы проектирования.docx
--------------------------------------------------


In [51]:
broken_files

[]

In [ ]:
# folder = '/srv/norma_ai/norma_ai_project/data_copy'

# for filename in broken_files:
#     file_path = os.path.join(folder, filename)
#     try:
#         if os.path.isfile(file_path):   # проверяем, что это файл, а не папка
#             os.remove(file_path)        # удаляем файл
#             print(f"Удалён: {file_path}")
#     except Exception as e:
#         print(f"Не удалось удалить {file_path}: {e}")

Удалён: /srv/norma_ai/norma_ai_project/data_copy/19.1 Приказ РТН № 444 ФНИП Правила промышленной безопасности при эксплуатации технологических трубопроводов.docx
Удалён: /srv/norma_ai/norma_ai_project/data_copy/11. Постановление Правительства № 1466  Об утверждении Правил подготовки, рассмотрения и согласования планов и схем развития горных работ .pdf
Удалён: /srv/norma_ai/norma_ai_project/data_copy/Федеральный закон от 31.07.2020 N 248-ФЗ О государственном контроле (надзоре) и муниципальном.rtf
Удалён: /srv/norma_ai/norma_ai_project/data_copy/19. Приказ РТН № 536 ФНИП Правила промышленной безопасности при использовании оборудования, работающего под избыточным давлением.rtf
Удалён: /srv/norma_ai/norma_ai_project/data_copy/7. ФЗ 116 О промышленной безопасности ОПО.docx
Удалён: /srv/norma_ai/norma_ai_project/data_copy/5. ФЗ 69 О пожарной безопасности.docx
Удалён: /srv/norma_ai/norma_ai_project/data_copy/СП 343.1325800.2017 Сооружения промышленных предприятий. Правила эксплуатации.pdf
Уда

In [49]:
len(loaded_docs)

0

In [44]:
# Проверочный блок для PDF

current_path = '/srv/norma_ai/norma_ai_project/data/3. ТР ТС 012-2011.pdf'
path = Path(current_path)
loaded_docs_test = PDFPlumberLoader(str(path)).load()

In [45]:
loaded_docs_test

[Document(metadata={'source': '/srv/norma_ai/norma_ai_project/data/3. ТР ТС 012-2011.pdf', 'file_path': '/srv/norma_ai/norma_ai_project/data/3. ТР ТС 012-2011.pdf', 'page': 0, 'total_pages': 21, 'Title': 'Решение Комиссии Таможенного союза от 18.10.2011 N 825(ред. от 01.12.2020)"О принятии технического регламента Таможенного союза "О безопасности оборудования для работы во взрывоопасных средах"(вместе с "ТР ТС 012/2011. Технический регламент Таможенного со', 'Author': 'Evgeniy Ostromecky', 'Creator': 'Microsoft® Word 2016', 'CreationDate': "D:20250826143355+03'00'", 'ModDate': "D:20250826143355+03'00'", 'Producer': 'Microsoft® Word 2016'}, page_content='Решение Комиссии Таможенного союза от\n18.10.2011 N 825\n(ред. от 01.12.2020)\n"О принятии технического регламента\nТаможенного союза "О безопасности\nоборудования для работы во взрывоопасных\nсредах"\n(вместе с "ТР ТС 012/2011. Технический\nрегламент Таможенного союза. О\nбезопасности оборудования для работы во\nвзрывоопасных средах")\

In [ ]:
# Проверочный код для DOCX

current_path = '/srv/norma_ai/norma_ai_project/data/9. ФЗ 384 О безопасности зданий и сооружений.docx'

Docx2txtLoader(current_path).load()

[Document(metadata={'source': '/srv/norma_ai/norma_ai_project/data/9. ФЗ 384 О безопасности зданий и сооружений.docx'}, page_content='Федеральный закон от 30.12.2009 N 384-ФЗ\n(ред. от 02.07.2013)\n"Технический регламент о безопасности зданий и сооружений"\n\nДокумент предоставлен КонсультантПлюс\nДата сохранения: 14.02.2022\n\n\n\n \n\n\n\nФедеральный закон от 30.12.2009 N 384-ФЗ\n(ред. от 02.07.2013)\n"Технический регламент о безопасности зданий и сооружений"\n\nДокумент предоставлен КонсультантПлюс\n\nwww.consultant.ru\n\nДата сохранения: 14.02.2022\n\xa0\n\n\n\n\n\n30\xa0декабря\xa02009\xa0года\n\nN\xa0384-ФЗ\n\n\n\n\n\nРОССИЙСКАЯ ФЕДЕРАЦИЯ\n\n\n\nФЕДЕРАЛЬНЫЙ ЗАКОН\n\n\n\nТЕХНИЧЕСКИЙ РЕГЛАМЕНТ О БЕЗОПАСНОСТИ ЗДАНИЙ И СООРУЖЕНИЙ\n\n\n\nПринят\n\nГосударственной Думой\n\n23 декабря 2009 года\n\n\n\nОдобрен\n\nСоветом Федерации\n\n25 декабря 2009 года\n\n\n\n\n\n\n\nСписок изменяющих документов\n\n(в ред. Федерального закона от 02.07.2013 N 185-ФЗ)\n\n\n\n\n\nГлава 1. ОБЩИЕ ПОЛОЖЕНИЯ\

In [52]:
# Чанкинг под регламенты (чуть крупнее блоки)
splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
docs = splitter.split_documents(loaded_docs)

In [53]:
# (опционально) добавить базовые метаданные
for d in docs:
    d.metadata.setdefault("source", d.metadata.get("source") or "local")

# Индексация в ту же коллекцию:
# vs = QdrantVectorStore(
#     client=client,
#     collection_name="norma_ai_collection",
#     embedding=embeddings_model
#     ).add_documents(docs)

vs = QdrantVectorStore(
    client=client,  # твой embedded Qdrant
    collection_name="norma_ai_collection",
    embedding=embeddings_model,
)


In [ ]:
# Индексация в ту же коллекцию:
# vs = QdrantVectorStore(
#     client=client,
#     collection_name="norma_ai_collection",
#     embedding=embeddings_model
#     ).add_documents(docs)

vs = QdrantVectorStore(
    client=client, 
    collection_name="norma_ai_collection_ru-en-RoSBERTa",
    embedding=embeddings_model_2,
)


In [108]:
vs.add_documents(docs)

['b498f2a3551242b5a1faa23cf66a008b',
 'c22c24ba32aa4634a672eb02ccedef6d',
 '20e28a9caa5844d9bd382968bc81f515',
 '8308d3da7f10418cbf3e35e5fe2948d8',
 '9d4d73c60bee42f0b59c59ecee317c94',
 '0fa89e4723784dbebf10eebdc78e8ce6',
 '852587e5d6434f7d8ff0b3ad3999eb2a',
 'dbfade332ce340cb893ddb65d55a1d84',
 '4e6913a3c99049bf82da68aa38689c41',
 '45d6156d4fec4a1da2151a0095120254',
 '35dbf31888184283880180cd2e96574f',
 '655f6b0bcbf84b5c84abb6bfbf5d9a52',
 'ae010143157f49978c6eb2c00e681d9c',
 '10522d8945dc48f4abbf0277392f02eb',
 'b7a61b754f9c4756b8ad02af0a5b848b',
 'cfc8f4cab66b4ff5b1da6dde84f1fd58',
 'fd73c1c3cec54ed7815b68502b0486d6',
 '64947e2142d5492d8928b7e7050c6d4a',
 'df63ae6e9f8b4a8389e6546ba82bb8e1',
 '16ba5613c35a4c7ea9531c462888b8af',
 'a7321c504bb441e7bae22fe17601e5e5',
 'a7f42743360442c486a76944d3431dd7',
 '3c730e22bb06418d9ca05179a6b21f92',
 '1f51dbae03b7464da8b21774507705b1',
 '2c6d890dad004f28948fc6d17f2059ce',
 '20cd3cc3039f454689efc49b4617cc41',
 '3f9355b11f3b4268970bb8e5dc157a99',
 

In [109]:
client.get_collections()


CollectionsResponse(collections=[CollectionDescription(name='norma_ai_collection'), CollectionDescription(name='norma_ai_collection_ru-en-RoSBERTa')])

In [114]:
client.count(collection_name=collection_2)

CountResult(count=8035)

In [118]:
query = "Периодичность ревизии промысловых трубопроводов"
res = vs.similarity_search(query, k=5)

In [119]:
res[0]

Document(metadata={'source': '/srv/norma_ai/norma_ai_project/data/25. Приказ РТН № 784 Рекомендации по устройству и безопасной эксплуатации технологических трубопроводов.docx', '_id': 'e82c9a7246f74c44a1099917cfb5b5f9', '_collection_name': 'norma_ai_collection_ru-en-RoSBERTa'}, page_content='430. Периодичность последующих освидетельствований рекомендуется устанавливать в зависимости от скорости коррозионно-эрозионного износа трубопроводов, условий эксплуатации, результатов предыдущих осмотров, ревизии и других факторов. Сроки ревизии рекомендуется определять предприятию - владельцу трубопровода и указывать в паспорте на трубопровод. В случае отсутствия таких указаний периодичности сроки проведения ревизии трубопроводов давлением до 10 МПа рекомендуется определять с учетом приложения N 22 к настоящему Руководству.\n\n431. При ревизии трубопровода с давлением до 10 МПа рекомендуется:\n\nпроводить наружный осмотр трубопровода согласно требованиям п. 428 настоящего Руководства;\n\nпроводит

In [120]:
for r in res:
    print(f'ДОКА: {r.metadata['source'][r.metadata['source'].rfind('/')+1:]}', r.page_content, sep='\n\n')# res[3].metadata['source'][r.metadata['source'].rfind('/')+1:])

ДОКА: 25. Приказ РТН № 784 Рекомендации по устройству и безопасной эксплуатации технологических трубопроводов.docx

430. Периодичность последующих освидетельствований рекомендуется устанавливать в зависимости от скорости коррозионно-эрозионного износа трубопроводов, условий эксплуатации, результатов предыдущих осмотров, ревизии и других факторов. Сроки ревизии рекомендуется определять предприятию - владельцу трубопровода и указывать в паспорте на трубопровод. В случае отсутствия таких указаний периодичности сроки проведения ревизии трубопроводов давлением до 10 МПа рекомендуется определять с учетом приложения N 22 к настоящему Руководству.

431. При ревизии трубопровода с давлением до 10 МПа рекомендуется:

проводить наружный осмотр трубопровода согласно требованиям п. 428 настоящего Руководства;

проводить измерения толщины стенки трубопровода приборами неразрушающего контроля, а в обоснованных случаях - сквозной засверловкой с последующей заваркой отверстия;

проводить радиографическ

In [16]:
points = client.scroll(
    collection_name="norma_ai_collection",
    limit=3,                     # сколько объектов вытащить
    with_payload=True,            # вернуть текст и метаданные
    with_vectors=True             # ВАЖНО: вернуть сами эмбеддинги
)

In [17]:
for p in points[0]:
    print(p.payload['page_content'])

б) строительный контроль (технический надзор), авторский и государственный надзор - при строительстве (реконструкции);

в) приемка - при завершении строительства (реконструкции);

г) эксплуатационный контроль, государственный контроль (надзор) - при эксплуатации;

д) строительный контроль и авторский надзор - при утилизации (ликвидации).

51. Порядок применения форм оценки соответствия магистрального трубопровода требованиям настоящего технического регламента устанавливается законодательством государств-членов.

52. При проведении оценки соответствия применяются средства измерений и методики (методы) измерений, являющиеся объектами государственного регулирования в области обеспечения единства измерений, в том числе государственного метрологического контроля (надзора), и соответствующие обязательным метрологическим и техническим требованиям, установленным законодательством государства-члена, на территории которого они применяются, а также положениям актов органов Союза в области обеспеч

In [18]:
dict(points[0][0])['payload']

{'page_content': 'б) строительный контроль (технический надзор), авторский и государственный надзор - при строительстве (реконструкции);\n\nв) приемка - при завершении строительства (реконструкции);\n\nг) эксплуатационный контроль, государственный контроль (надзор) - при эксплуатации;\n\nд) строительный контроль и авторский надзор - при утилизации (ликвидации).\n\n51. Порядок применения форм оценки соответствия магистрального трубопровода требованиям настоящего технического регламента устанавливается законодательством государств-членов.\n\n52. При проведении оценки соответствия применяются средства измерений и методики (методы) измерений, являющиеся объектами государственного регулирования в области обеспечения единства измерений, в том числе государственного метрологического контроля (надзора), и соответствующие обязательным метрологическим и техническим требованиям, установленным законодательством государства-члена, на территории которого они применяются, а также положениям актов орг